In [1]:
import sqlite3
import pandas as pd
import numpy as np
file=open("log.txt", "w")
# Conectar a la base de datos
conn = sqlite3.connect('DRUG_DB.db')
cursor = conn.cursor()

# Obtener los nombres de las tablas (drogas)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()
cont=0
# Iterar a través de las tablas
for table_name in table_names:
    table_name = table_name[0]
    
    # Obtener los parámetros únicos en esta tabla
    params_query = f"""SELECT DISTINCT Parameter FROM "{table_name}" """
    unique_params = [row[0] for row in cursor.execute(params_query).fetchall()]
    
    # Iterar a través de los parámetros únicos
    for param in unique_params:
        # Obtener los valores asociados a este parámetro en la tabla
        values_query = f"""SELECT Value FROM "{table_name}" WHERE Parameter = "{param}" AND Accepted_value = 1 AND Accepted_unit = 1 """
        values = [row[0] for row in cursor.execute(values_query).fetchall()]



        cursor.execute(f"""PRAGMA table_info("{table_name}")""")
        existing_columns = [column_info[1] for column_info in cursor.fetchall()]

        if 'mean' not in existing_columns:
            alter_query = f"""ALTER TABLE "{table_name}" ADD COLUMN mean REAL"""
            cursor.execute(alter_query)

        if 'std' not in existing_columns:
            alter_query = f"""ALTER TABLE "{table_name}" ADD COLUMN std REAL"""
            cursor.execute(alter_query)

        if 'median' not in existing_columns:
            alter_query = f"""ALTER TABLE "{table_name}" ADD COLUMN median REAL"""
            cursor.execute(alter_query)


        # Filtrar los valores None del arreglo
        filtered_values = [value for value in values if value is not None]

        # Calcular la media y desviación estándar solo si hay valores válidos
        if len(filtered_values) > 0:
            numeric_values = []
            
            for value in filtered_values:
    
                try:
                    numeric_values.append(float(value))
                except:
                    cont=cont+1
                    file.write(f'{value}\n')
                    print(value)
                    pass
            mean_value = np.mean(numeric_values)
            std_dev = np.std(numeric_values)
            median_value = np.median([float(val) for val in numeric_values])
        else:
            mean_value = None
            std_dev = None
            median_value = None
        
        # Actualizar la tabla añadiendo las columnas "media" y "desviacion"
        update_query = f"""UPDATE "{table_name}" SET mean = ?, median = ?, std = ? WHERE Parameter = ?"""
        cursor.execute(update_query, (mean_value, median_value, std_dev, param))
    
    # Guardar los cambios en la base de datos
    conn.commit()

# Cerrar la conexión a la base de datos
conn.close()
file.close()
print(cont)


0
